In [1]:
"""
读取指定文件夹中的多个日志文件，解析每个日志文件中的训练指标，并计算多次训练的平均训练指标。最终将平均训练指标写入一个txt文件中
"""
import os
from glob import glob

logs_dir = r'E:\训练作业输出\5.22 finetune 100B surveillance\rawLog'

logs_name = os.listdir(logs_dir)

save_txt = open(logs_dir + '.txt', 'w')
result_dict = {}

In [2]:
logs_name

['modelarts-job-5ef201d4-ead5-483e-a094-705a1cec369d-worker-0.log',
 'modelarts-job-5ef201d4-ead5-483e-a094-705a1cec369d-worker-1.log',
 'modelarts-job-5ef201d4-ead5-483e-a094-705a1cec369d-worker-10.log',
 'modelarts-job-5ef201d4-ead5-483e-a094-705a1cec369d-worker-11.log',
 'modelarts-job-5ef201d4-ead5-483e-a094-705a1cec369d-worker-12.log',
 'modelarts-job-5ef201d4-ead5-483e-a094-705a1cec369d-worker-13.log',
 'modelarts-job-5ef201d4-ead5-483e-a094-705a1cec369d-worker-14.log',
 'modelarts-job-5ef201d4-ead5-483e-a094-705a1cec369d-worker-15.log',
 'modelarts-job-5ef201d4-ead5-483e-a094-705a1cec369d-worker-2.log',
 'modelarts-job-5ef201d4-ead5-483e-a094-705a1cec369d-worker-3.log',
 'modelarts-job-5ef201d4-ead5-483e-a094-705a1cec369d-worker-4.log',
 'modelarts-job-5ef201d4-ead5-483e-a094-705a1cec369d-worker-5.log',
 'modelarts-job-5ef201d4-ead5-483e-a094-705a1cec369d-worker-6.log',
 'modelarts-job-5ef201d4-ead5-483e-a094-705a1cec369d-worker-7.log',
 'modelarts-job-5ef201d4-ead5-483e-a094-70

In [3]:

for log in logs_name:
    path = os.path.join(logs_dir, log)
    result_dict[log] = {'acc1': [], 'acc5': [], 'loss': []}
    with open(path, 'r', encoding='utf-8') as f:
        log_info = f.readlines()

        count_num = 0
        acc1, acc5, loss = 0, 0, 0
        for line in log_info:
            if 'acc1' in line and 'acc5' in line and 'eval_loss' in line:
                acc1_ = float(line.split('\'acc1\': ')[1].split(', \'acc5\'')[0])
                acc5_ = float(line.split('\'acc5\': ')[1].split(', \'eval_loss\'')[0])
                eval_loss_ = float(line.split('\'eval_loss\': ')[1].split('}')[0])

                if count_num % 8 == 0 and count_num != 0:
                    acc1 = acc1_
                    acc5 = acc5_
                    loss = eval_loss_
                else:
                    acc1 += acc1_
                    acc5 += acc5_
                    loss += eval_loss_

                count_num += 1
                if count_num % 8 == 0 and count_num > 0:
                    result_dict[log]['acc1'].append(acc1 / 8)
                    result_dict[log]['acc5'].append(acc5 / 8)
                    result_dict[log]['loss'].append(loss / 8)

    acc1_result = []
    acc5_result = []
    loss_result = []
    for log in logs_name:
        acc1_result.append(result_dict[log]['acc1'])
        acc5_result.append(result_dict[log]['acc5'])
        loss_result.append(result_dict[log]['loss'])

    import numpy as np

    acc1_result_np = np.array(acc1_result)
    acc5_result_np = np.array(acc5_result)
    loss_result_np = np.array(loss_result)
    acc1_mean = np.mean(acc1_result_np, axis=0)
    acc5_mean = np.mean(acc5_result_np, axis=0)
    loss_mean = np.mean(loss_result_np, axis=0)

    for acc1, acc5, loss in zip(acc1_mean, acc5_mean, loss_mean):
        save_txt.write(f'acc1:{acc1},acc5:{acc5},eval_loss:{loss}\n')

KeyError: 'modelarts-job-5ef201d4-ead5-483e-a094-705a1cec369d-worker-1.log'